## Name Commonness

Standard string similarity measures treat all values equally — "Smith" matching "Smith" scores the same as "Ximénez-Fatio" matching "Ximénez-Fatio". In practice, matching on rare values is far more informative than matching on common ones. The `Commonness` class addresses this by computing frequency-based commonness scores for specified variables and registering a custom `commonness_score` similarity function that rewards rare-and-equal matches.

For each variable, the class computes how common each value is in a reference corpus and appends a `<variable>_commonness` column (values in [0, 1]) to both DataFrames. The custom similarity function scores pairs as `(1 - |x - y|) * (1 - mean(x, y))`, so identical rare values score high and identical common values score low.

Commonness scores should be computed *after* data harmonization (`Prepare.format()`) but *before* training, so that the `_commonness` columns are available as features.

The `df_left_full` and `df_right_full` parameters define the full datasets used for frequency estimation. If the training data is representative of the population, you can pass the training DataFrames themselves (i.e., `df_left_full=left` and `df_right_full=right`). If a larger or more complete dataset is available, passing it will yield more reliable frequency estimates.


In [ ]:
from neer_match_utilities.prepare import Commonness

c = Commonness(
    variable_list=['name', 'surname'],
    df_left=left,
    df_right=right,

    # Reference corpus for frequency estimation
    df_left_full=left_full,       # full left dataset (or left if representative)
    df_right_full=right_full,     # full right dataset (or right if representative)

    commonness_source='both',     # "left" | "right" | "both" — which corpus to use
    scoring='minmax',             # "relative" | "minmax" | "log"
    fill_value=0.0,               # score for unseen values
    preprocess=True,              # normalize strings (strip & lowercase) before counting
)

left, right = c.calculate()

After calling `calculate()`, the DataFrames contain new columns (e.g., `name_commonness`, `surname_commonness`) that can be included in the `similarity_map` using the `commonness_score` similarity concept.

## Stop Word Removal with spaCy

The `Prepare` class supports removing stop words from string variables using a [spaCy](https://spacy.io/) language pipeline. Stop words are common words (e.g., "the", "and", "of") that carry little meaning for entity matching and can reduce similarity scores between otherwise matching records.

To enable stop word removal, pass a spaCy pipeline name to `spacy_pipeline`. The pipeline provides a language-specific list of stop words. spaCy models are available in more than 20 languages — see [spacy.io/models](https://spacy.io/models) for the full list. You can also specify `additional_stop_words` to remove domain-specific terms that are frequent but uninformative for matching (e.g., legal forms like "AG", "GmbH"). Stop words are only removed when `remove_stop_words=True` is passed to `prepare.format()`.

Note that if the similarity map includes numeric similarity concepts, the corresponding columns must have a numeric dtype. The `to_numeric` argument in `prepare.format()` ensures this by converting the specified columns, which is useful when numeric data was read as strings (e.g., from CSV files).


In [ ]:
prepare = Prepare(
    similarity_map=similarity_map,
    df_left=left,
    df_right=right,
    id_left='company_id',
    id_right='company_id',
    spacy_pipeline='de_core_news_sm',
    additional_stop_words=['AG']
)

# Get formatted and harmonized datasets

left, right = prepare.format(
    fill_numeric_na=False,
    to_numeric=['found_year'],
    fill_string_na=True,
    capitalize=True,
    lower_case=False,
    remove_stop_words=True,
)

## Feature Selection

The `FeatureSelector` allows you to start with a large similarity map — many feature pairs and many similarity concepts per pair — to maximize potential performance, and then automatically reduce it to the most informative subset. It uses a two-stage procedure:

1. **Stage 1 — Correlation filtering** (optional): Groups highly correlated features and keeps only the one most correlated with the target variable. This removes redundant features before regularization.
2. **Stage 2 — Elastic net regularization**: Fits a penalized logistic regression (L1/L2 mix) with cross-validation to select features that contribute unique predictive information. Features with zero or near-zero coefficients are dropped.

The selector is designed for the extreme class imbalance typical in entity matching, where true matches are rare relative to non-matches.


In [ ]:
from neer_match_utilities.feature_selection import FeatureSelector

fs = FeatureSelector(
    similarity_map=similarity_map,
    training_data=(left_train, right_train, matches_train),

    # ID and match columns
    id_left_col="id_unique",
    id_right_col="id_unique",
    matches_id_left="left",
    matches_id_right="right",
    match_col="match",
    matches_are_indices=True,

    # Stage 1: Correlation filtering
    max_correlation=0.95,       # drop features with pairwise correlation > 0.95

    # Stage 2: Elastic net
    scoring="average_precision", # metric for CV; recommended for imbalanced data
    cv=2,                        # number of cross-validation folds
    Cs=20,                       # number of regularization strengths to search
    class_weight="balanced",     # adjust for class imbalance
    min_coef_threshold=0.01,     # drop features with |coefficient| below this

    random_state=42,
    n_jobs=4,
)

fs_result = fs.execute()

The result object contains the reduced similarity map, which can be used directly in subsequent training:


In [ ]:
# Updated similarity map with only selected features
similarity_map = fs_result.updated_similarity_map

# Inspect feature importance via coefficients
print(fs_result.coef_by_feature)

# Check selection metadata
print(f"Features: {fs_result.meta['n_features_in']} → {fs_result.meta['n_features_selected']}")

## Handling Many-to-Many Matches

### Loading the Data

First, we import the necessary libraries and load the datasets.


In [ ]:
#| eval: true
import random
import pandas as pd

matches = pd.read_csv('matches.csv')
left = pd.read_csv('left.csv')
right = pd.read_csv('right.csv')

### Inspecting the Data

The first few rows of each dataset show their structure.


In [ ]:
#| eval: true
matches.head()

In [ ]:
#| eval: true
left.head()

In [ ]:
#| eval: true
right.head()

All three DataFrames have the same number of observations:


In [ ]:
#| eval: true
print(f'Number of observations in matches: {len(matches)}')
print(f'Number of observations in left: {len(left)}')
print(f'Number of observations in right: {len(right)}')

### Simulating a Many-to-Many Relationship

To demonstrate how to handle more complex matching scenarios, we simulate a many-to-many (m:m) relationship. Assume that the company with `company_id` *1e87fc75b4* in the *left* DataFrame should match with two entries in the *right* DataFrame: the original match *0008e07878* and an additional match *8bf51ba8a0*.


In [ ]:
#| eval: true
# Add an extra match to simulate a many-to-many relationship

extra_match = pd.DataFrame({
    'company_id_left' : ['1e87fc75b4'],
    'company_id_right' : ['8bf51ba8a0']
})
matches = pd.concat([matches, extra_match], ignore_index=True)

Now, inspect the modified `matches` dataframe for the affected IDs:


In [ ]:
#| eval: true
matches[
    matches['company_id_left'].isin(['1e87fc75b4', '810c9c3435']) |
    matches['company_id_right'].isin(['0008e07878', '8bf51ba8a0'])
]

### Understanding the Matching Issue

Simply adding a new row to the *matches* DataFrame can be problematic. Consider this simplified example:

| Left | Right | Implied Real-World Entity |
|------|-------|---------------------------|
| A    | C     | Entity 1                  |
| B    | D     | Entity 2                  |

If further evidence shows that record *A* and record *C* represent the same entity, then all related records (*A*, *B*, *C*, *D*) should be grouped together. This grouping implies that every possible pair among these records should be represented (as shown in the first six rows of the table below). The observations *B* and *C* would consequently appear in both the *Left* and *Right* columns, so the *left* and *right* DataFrames need to be adjusted to include these observations in both. The *matches* DataFrame must be expanded with additional entries (highlighted by the <span style="color: orange;">orange</span> rows):

| Left | Right |
|------|-------|
| A    | B     |
| A    | C     |
| A    | D     |
| B    | C     |
| B    | D     |
| C    | D     |
| <span style="color: orange;">B</span>    | <span style="color: orange;">B</span>     |
| <span style="color: orange;">C</span>    | <span style="color: orange;">C</span>     |

This example highlights why a naive approach of merely adding an extra match does not fully capture the nature of the linking problem.

### Correcting the Relationships

To correctly group all records representing the same real-world entity, we use the `data_preparation_cs` method from the `SetupData` class. This method automatically completes the matching pairs and adjusts the `left` and `right` datasets accordingly.


In [ ]:
#| eval: true
from neer_match_utilities.panel import SetupData

left, right, matches = SetupData(matches=matches).data_preparation_cs(
    df_left=left,
    df_right=right,
    unique_id='company_id'
)

### 6. Verifying the Adjustments

Finally, we verify that the adjustments correctly reflect the intended relationships by checking the relevant company IDs in the updated datasets.


In [ ]:
#| eval: true
# Verify the updated matches for the specific company_ids

artificial_group = ['1e87fc75b4', '810c9c3435', '0008e07878', '8bf51ba8a0']

matches_subset = matches[
    matches['left'].isin(artificial_group) |
    matches['right'].isin(artificial_group)
].sort_values(['left', 'right'])

matches_subset

In [ ]:
#| eval: true
# Check the corresponding records in the left dataset

left_subset = left[
    left['company_id'].isin(artificial_group)
][['company_id']]
left_subset.head(10)

In [ ]:
#| eval: true
# Check the corresponding records in the right dataset

right_subset = right[
    right['company_id'].isin(artificial_group)
][['company_id']]
right_subset

These steps ensure that the data accurately represents the underlying real-world relationships, even when the matching is more complex than a simple 1:1 mapping. To prevent the simulated change from affecting subsequent steps, we drop the observations associated with these IDs.


In [ ]:
#| eval: true
left = left[~left['company_id'].isin(artificial_group)].reset_index(drop=False)
right = right[~right['company_id'].isin(artificial_group)].reset_index(drop=False)
matches = matches[
    (~matches['left'].isin(artificial_group))
    &
    (~matches['right'].isin(artificial_group))
].reset_index(drop=False)